In [ ]:
# usare object_detection vecchio (in locale)
!git clone https://github.com/tensorflow/models.git
!apt-get -qq install libprotobuf-java protobuf-compiler
!protoc ./models/research/object_detection/protos/string_int_label_map.proto --python_out=.
!cp -R models/research/object_detection/ object_detection/
!rm -rf models

In [ ]:
!pip3 install numpy
!pip3 install tensorflow-gpu
!pip3 install opencv-contrib-python
!pip3 install matplotlib
!pip3 install Pillow
!pip3 install scipy
!pip3 install opencv-python
!apt update && apt install -y libsm6 libxext6
!apt-get install -y libxrender-dev

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from scipy.stats import itemfreq

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
!mkdir training
!mkdir annotations
!mkdir pre-trained-model
!mkdir images

!mkdir images/test
!mkdir images/train

In [ ]:
MODEL_NAME = 'ssd_mobilenet_v2_coco_2018_03_29'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('annotations', 'label_map.pbtxt')
NUM_CLASSES = 1

# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())
    
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
!git clone https://github.com/tensorflow/models.git
!apt-get -qq install libprotobuf-java protobuf-compiler
!protoc ./models/research/object_detection/protos/string_int_label_map.proto --python_out=.
!cp -R models/research/slim/nets nets/
!rm -rf models

!git clone https://github.com/tensorflow/models.git
!apt-get -qq install libprotobuf-java protobuf-compiler
!protoc ./models/research/object_detection/protos/string_int_label_map.proto --python_out=.
!cp -R models/research/slim/deployment deployment/
!rm -rf models

In [ ]:
#!pip3 install unrar
#!unrar x object_detection.rar
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:


from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

!nvidia-smi
get_available_gpus()
#device_lib.list_local_devices()
# !pip install --no-index -f https://dist.plone.org/thirdparty/ -U PIL
# !pip install Pillow
#!apt install -y nvidia-387
#!python3 --version

In [ ]:
#!nvidia -smi
# !lspci | grep -i nvidia
#!apt-cache search nvidia | grep -E "nvidia\-[0-9]{3}"
#!dpkg -l | grep -E "nvidia-[0-9]{3}"
# !apt-cache policy nvidia-410

#! add-apt-repository -y ppa:graphics-drivers/ppa                          
#!apt install -y nvidia-410  
#!dpkg --configure -a
#!apt-get -f install

# Assume that you have 12GB of GPU memory and want to allocate ~4GB:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [ ]:
with tf.device('/gpu:1'):
    !CUDA_VISIBLE_DEVICES=1 python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v2_coco.config

In [ ]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v2_coco.config --trained_checkpoint_prefix training/model.ckpt-81646 --output_directory trained-inference-graphs/output_inference_graph_81k

In [ ]:
# da installare:
# !pip3 install cython
# !pip3 install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools^&subdirectory=PythonAPI"

# !python eval.py --logtostderr \
# --train_dir=training \
# --pipeline_config_path=training/ssd_mobilenet_v2.config \
# --checkpoint_dir=training --eval_dir=evaluation \
!ls images/train | wc -l